In [ ]:
pip install cryptography

In [ ]:
import hashlib
import getpass
import json
import os
import base64
from cryptography.fernet import Fernet

class PasswordManager:
    def __init__(self, master_password):
        # Derive a key from the master password for more secure encryption
        key = hashlib.pbkdf2_hmac('sha256', master_password.encode('utf-8'), b'salt', 100000)
        self.master_password_hash = hashlib.sha256(master_password.encode()).hexdigest()
        
        # Initialize cipher for encrypting passwords
        self.cipher = Fernet(base64.urlsafe_b64encode(key))

        # Dictionary to store passwords
        self.passwords = {}

    def _encrypt(self, data):
        return self.cipher.encrypt(data.encode()).decode()

    def _decrypt(self, encrypted_data):
        return self.cipher.decrypt(encrypted_data.encode()).decode()

    def _hash_password(self, password):
        # Using SHA-256 for password hashing
        return hashlib.sha256(password.encode()).hexdigest()

    def _check_password_strength(self, password):
        # Simple password strength check (you can enhance this based on requirements)
        if len(password) < 8:
            return False
        return True

    def save_password(self, account, password):
        if self._hash_password(getpass.getpass("Enter master password: ")) == self.master_password_hash:
            if self._check_password_strength(password):
                encrypted_password = self._encrypt(password)
                self.passwords[account] = encrypted_password
                print("Password saved successfully!")
            else:
                print("Password is not strong enough. Please choose a stronger password.")
        else:
            print("Incorrect master password. Operation aborted.")

    def get_password(self, account):
        if account in self.passwords:
            if self._hash_password(getpass.getpass("Enter master password: ")) == self.master_password_hash:
                decrypted_password = self._decrypt(self.passwords[account])
                print(f"Password for {account}: {decrypted_password}")
            else:
                print("Incorrect master password. Operation aborted.")
        else:
            print(f"No password found for {account}.")

    def export_passwords(self, filename):
        if self._hash_password(getpass.getpass("Enter master password: ")) == self.master_password_hash:
            data = {
                "master_password_hash": self.master_password_hash,
                "passwords": self.passwords
            }

            with open(filename, "w") as file:
                json.dump(data, file)

            print(f"Passwords exported to {filename} successfully!")
        else:
            print("Incorrect master password. Operation aborted.")

    def import_passwords(self, filename):
        if self._hash_password(getpass.getpass("Enter master password: ")) == self.master_password_hash:
            if os.path.exists(filename):
                with open(filename, "r") as file:
                    data = json.load(file)

                if data["master_password_hash"] == self.master_password_hash:
                    self.passwords = data["passwords"]
                    print(f"Passwords imported from {filename} successfully!")
                else:
                    print("Incorrect master password hash in the file. Operation aborted.")
            else:
                print(f"File {filename} does not exist.")
        else:
            print("Incorrect master password. Operation aborted.")

if __name__ == "__main__":
    master_password = getpass.getpass("Set master password: ")
    password_manager = PasswordManager(master_password)

    while True:
        print("\nPassword Manager Menu:")
        print("1. Save Password")
        print("2. Get Password")
        print("3. Export Passwords")
        print("4. Import Passwords")
        print("0. Exit")
        
        choice = input("Enter your choice: ")

        if choice == "1":
            account = input("Enter account name: ")
            password = getpass.getpass("Enter password: ")
            password_manager.save_password(account, password)
        elif choice == "2":
            account = input("Enter account name: ")
            password_manager.get_password(account)
        elif choice == "3":
            filename = input("Enter export filename: ")
            password_manager.export_passwords(filename)
        elif choice == "4":
            filename = input("Enter import filename: ")
            password_manager.import_passwords(filename)
        elif choice == "0":
            print("Exiting Password Manager. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter a valid option.")


Set master password: ········

Password Manager Menu:
1. Save Password
2. Get Password
3. Export Passwords
4. Import Passwords
0. Exit
